In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow import keras
from skimage.util import random_noise

In [ ]:

# def noise(array):
#     """
#     Adds random noise to each image in the supplied array.
#     """

#     noise_factor = 0.1
#     noisy_array = array + noise_factor * np.random.normal(
#         loc=0.0, scale=1.0, size=array.shape
#     )

#     return np.clip(noisy_array, 0.0, 1.0)
import random

def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = random.randrange(0, 40)/100
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    array = np.clip(noisy_array, 0.0, 1.0)
   
    std = random.randrange(0, 40)/100

    noisy_images = [img + np.random.normal(0, std, img.shape) for img in array]

    return noisy_images


def show_image(img):
    plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()




In [ ]:
# Global settings
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128

from morse_helpers import create_sets
from morse_label_funcs import  labels_to_one_hot_positions_categorical, letter_n_to_one_hot_positions_categorical, position_regression


def get_sets():
    return [
        # ["./training_data/MorseTrainSet_04/GEN04_VER_000/", 'wordsMatrices_04_000', "Words_04_000.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_100/", 'wordsMatrices_04_100', "Words_04_100.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_200/", 'wordsMatrices_04_200', "Words_04_200.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_210/", 'wordsMatrices_04_210', "Words_04_210.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_220/", 'wordsMatrices_04_220', "Words_04_220.csv"],

        # ["./training_data/MorseTrainSet_11/GEN11_VER_000/", 'wordsMatrices_11_000', "Words_11_000.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_100/", 'wordsMatrices_11_100', "Words_11_100.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_200/", 'wordsMatrices_11_200', "Words_11_200.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_210/", 'wordsMatrices_11_210', "Words_11_210.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_220/", 'wordsMatrices_11_220', "Words_11_220.csv"],

        # ["./training_data/MorseTrainSet_06/GEN06_VER_000/", 'wordsMatrices_06_000', "Words_06_000.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_100/", 'wordsMatrices_06_100', "Words_06_100.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_200/", 'wordsMatrices_06_200', "Words_06_200.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_210/", 'wordsMatrices_06_210', "Words_06_210.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_220/", 'wordsMatrices_06_220', "Words_06_220.csv"],

        ["./training_data/MorseTrainSet_13/GEN13_VER_000/", 'wordsMatrices_13_000', "Words_13_000.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_100/", 'wordsMatrices_13_100', "Words_13_100.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_200/", 'wordsMatrices_13_200', "Words_13_200.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_210/", 'wordsMatrices_13_210', "Words_13_210.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_220/", 'wordsMatrices_13_220', "Words_13_220.csv"],

    ]

(image_fnames, morse_labels) = create_sets(
    get_sets(), 
    IMAGE_TARGET_SIZE,
    [position_regression, letter_n_to_one_hot_positions_categorical],
    letter_n=1,
    overwrite_images=False
)

In [ ]:
# Concat label arrays
morse_labels_concat = np.array([morse_labels[0], morse_labels[1].astype("int")]).T

In [ ]:
from morse_helpers import create_all_sets

TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90

train, labels, train_validation, labels_validation, train_test, labels_test = create_all_sets(
    image_fnames, morse_labels_concat, TEST_SPLIT_SIZE, VALIDATION_SPLIT_SIZE, shuffle_before_test_split=True)


In [ ]:
from morse_helpers import convert_image_to_array
class Image_Generator_AutoEncoder(keras.utils.Sequence) :
    
    def __init__(self, image_filenames, labels, batch_size, image_target_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.image_target_size = image_target_size
        # self.image_prepocessors = image_prepocessors
        # self.label_func = label_func
        
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

    def __get_labels__(self, idx): 
         batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
         return batch_y
    
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        train_image_lists = []
        for img_name in batch_x:
            img = convert_image_to_array(img_name, self.image_target_size)
            train_image_lists.append(img)

        n = random.randint(0, 20)
        train_image_lists = [np.pad(img, [(0,0),(n,0), (0,0)], mode='constant')[:, :1400] for img in train_image_lists]

        noisy = noise(np.array(train_image_lists))

        arrays = np.array(noisy)[:,0:-1, :1400] , np.array(train_image_lists)[:,0:-1, :1400]
        return arrays

In [ ]:
from morse_helpers import add_zeropad_random

image_prepocessors = [{"func" :add_zeropad_random, "params": [-10, 15]}]

training_batch_generator = Image_Generator_AutoEncoder(train, labels, BATCH_SIZE, IMAGE_TARGET_SIZE)
validation_batch_generator = Image_Generator_AutoEncoder(train_validation, labels_validation, BATCH_SIZE, IMAGE_TARGET_SIZE)

# t, l = training_batch_generator.__getitem__(0)


# for i in range(5):
#     show_image(t[i])




In [ ]:
input = layers.Input(shape=(4, 1400, 1))

# cropped = keras.layers.Cropping2D(cropping=((0, 1), (0,0)), data_format=None)(input)

# Encoder
x = layers.Conv2D(70, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# x = layers.BatchNormalization()(x)

x = layers.Conv2D(70, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((1, 2), padding="same")(x)

# x = layers.BatchNormalization()(x)

x = layers.Conv2D(70, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((1, 2), padding="same")(x)

# x = layers.BatchNormalization()(x)

# Decoder

x = layers.Conv2D(70, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((1, 2))(x)

x = layers.Conv2D(70, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((1, 2))(x)

x = layers.Conv2D(70, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

# x = layers.Conv2DTranspose(32, (3, 3), strides=1, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy", metrics=["mean_squared_error"])
autoencoder.summary()

init_epoch = 0

In [ ]:
# predictions = autoencoder.predict(test_data)
# display(test_data, predictions)

In [ ]:
# autoencoder.fit(
#     x=noisy_train_data,
#     y=train_data,
#     epochs=10,
#     batch_size=128,
#     shuffle=True,
#     validation_data=(noisy_test_data, test_data),
# )

NUM_EPOCHS = 15

def fit_model(epochs):
	
	global init_epoch
	history = autoencoder.fit(
					   training_batch_generator,
	                   steps_per_epoch = int(len(train) // BATCH_SIZE),
	                   epochs = epochs + init_epoch,
					   initial_epoch=init_epoch,
	                   verbose =1,
	                   validation_data = validation_batch_generator,
	                   validation_steps = int(len(train_validation) // BATCH_SIZE))
	
	init_epoch += epochs
	return history

history = fit_model(NUM_EPOCHS)


In [ ]:
from morse_helpers import Image_Generator


def add_noise(params):
    def add_noise(train_images, lbls, image_target_size):
        train_images = np.array(train_images)
        noise_factor = 0.20
        noisy_array = train_images + noise_factor * np.random.normal(
            loc=0.0, scale=1.0, size=train_images.shape
        )

        array = np.clip(noisy_array, 0.0, 1.0)
    
        std = 0.2
        noisy_images = [img + np.random.normal(0, std, img.shape) for img in array]

        return (noisy_images, lbls)
    
    return add_noise


def labels_to_one_hot(lbls_pos, lbls_letters):
    label_letters = lbls_letters.astype("int")
    # clean up magic numbers
    labels_one_hot = np.zeros((label_letters.size, 26))
    labels_one_hot[np.arange(label_letters.size),label_letters] = 1
    return labels_one_hot

def cut(params):
    def cut_and_center(train_images, lbls, image_target_size):

        cut_images = []
        # train_images = train_images.copy()

        for i in range(len(train_images)):
            train_images[i][:, int(lbls[i] * 1400) + 3:] = 0
            padding = int(150 / 2)  - int((int(lbls[i] *1400) + 3) / 2)
            train_image_padded = np.pad(train_images[i], [(0,0),(padding,0), (0,0)], mode='constant')[:, :1400]
            cut_images.append(train_image_padded)

        return (cut_images, lbls)
    return cut_and_center

def add_zeros_row(params):
    def add_zeros(train_images, lbls, image_target_size):
        z = np.zeros(1400)
        zreshape = z.reshape(1, 1400, 1)
        zadded = []
        for img in train_images:
            
            zadded.append(np.vstack((img, zreshape)))
        
        return (np.array(zadded), lbls)

    return add_zeros

def autoencode_array(params):
    def autoencode_array(train_images, lbls, image_target_size):

        autoencoded = autoencoder.predict(np.array(train_images))
        
        return  (autoencoded, lbls)

    return autoencode_array

def shrink_image(params):
    def shrink_image(train_images, lbls, image_target_size):
        
        return  (np.array(train_images)[:,0:-1, :1400], lbls)

    return shrink_image


image_prepocessors = [
    {"func" :add_noise, "params": ""}, 
    {"func" :shrink_image, "params": ""},  
    {"func" :autoencode_array, "params": ""},
    {"func" :add_zeros_row, "params": ""}, 
    {"func" :cut, "params": ""}
]
test_batch_generator_categories = Image_Generator(train_test, labels_test, BATCH_SIZE, IMAGE_TARGET_SIZE, image_prepocessors, labels_to_one_hot)



In [ ]:
loaded_model_3 = tf.keras.models.load_model("saved_model_3")

loaded_model_3.evaluate(test_batch_generator_categories, verbose=0)



In [ ]:
t, l = test_batch_generator_categories.__getitem__(0)

for i in range(50):

    show_image(t[i])
    print(l[i])